In [1]:
import torch
from diffusers import HunyuanVideoPipeline, HunyuanVideoTransformer3DModel, BitsAndBytesConfig

In [3]:
model_id = "hunyuanvideo-community/HunyuanVideo"

quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)
transformer = HunyuanVideoTransformer3DModel.from_pretrained(
    model_id,
    subfolder="transformer",
    quantization_config=quantization_config,
    torch_dtype=torch.bfloat16,
)


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [4]:
pipe = HunyuanVideoPipeline.from_pretrained(
    model_id, 
    transformer=transformer, 
    torch_dtype=torch.float16
)
pipe.vae.enable_tiling()        # For additional memory optimization
pipe.enable_model_cpu_offload() # If needed on smaller GPUs

output = pipe(
    prompt="A cat walks on the grass, realistic",
    height=1920,
    width=1080,
    num_frames=450,          # ~4 sec at 15 fps
    num_inference_steps=30
).frames

model_index.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

Fetching 20 files:   0%|          | 0/20 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/766 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/419 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/986M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

In [8]:
import imageio
import numpy as np


# Convert the frames to a list of numpy arrays
frames_np = [np.array(frame) for frame in output]

# Save the frames as a video file
video_path = "output_video.mp4"
imageio.mimsave(video_path, frames_np, fps=15) # Adjust fps as needed

# Display the video
from IPython.display import Video
Video(video_path, embed=True)

/opt/conda/envs/hunyuan_env/lib/python3.10/site-packages/imageio/core/util.py:508: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


ValueError: Could not find a backend to open `output_video.mp4`` with iomode `wI`.
Based on the extension, the following plugins might add capable backends:
  FFMPEG:  pip install imageio[ffmpeg]
  pyav:  pip install imageio[pyav]

In [12]:
import imageio

video_path = "cat_walking.mp4"
fps = 15

# output is already a list of frames
imageio.mimsave(video_path, output, fps=fps)
print(f"Video saved to {video_path}")


ValueError: Image must have 1, 2, 3 or 4 channels

In [25]:
from IPython.display import Video
Video("cat_walking.mp4", embed=True)